In [3]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast

#Loading AG News dataset and tokenizer:
dataset = load_dataset("ag_news")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

#Tokenization:
def preprocess(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
train_data = tokenized_dataset["train"].select(range(5000))
eval_data = tokenized_dataset["test"].select(range(1000))

In [7]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,
    logging_dir="./logs",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=157, training_loss=0.45708242039771596, metrics={'train_runtime': 2825.0248, 'train_samples_per_second': 1.77, 'train_steps_per_second': 0.056, 'total_flos': 165590154240000.0, 'train_loss': 0.45708242039771596, 'epoch': 1.0})

In [9]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

trainer.compute_metrics = compute_metrics
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.3280773460865021, 'eval_accuracy': 0.896, 'eval_f1': 0.894795277032651, 'eval_runtime': 174.9706, 'eval_samples_per_second': 5.715, 'eval_steps_per_second': 0.091, 'epoch': 1.0}
